# TAAC2019
>往届比赛分析

腾讯效果广告采用的是 GSP（Generalized Second-Price）竞价机制，广告的实际曝光取决于广告的流量覆盖大小和在竞争广告中的相对竞争力水平。其中广告的流量覆盖取决于广告的人群定向（匹配对应特征的用户数量）、广告素材尺寸（匹配的广告位）以及投放时段、预算等设置项。而影响广告竞争力的主要有出价、广告质量等因素（如 pctr/pcvr 等），以及对用户体验的控制策略。通常来说，基本竞争力可以用千次曝光收益`ecpm = 1000 *cpc_bid * pctr = 1000 * cpa_bid * pctr * pcvr (cpc, cpa 分别代表按点击付费模式和按转化付费模式)`。

综上，其中前者决定广告能参与竞争的次数以及竞争对象，后者决定在每次竞争中的胜出概率。二者最终决定广告每天的曝光量。
## 数据准备
[数据集下载_algo.qq.com_641013010_testa.zip](https://algo-data-1256087447.file.myqcloud.com/algo.qq.com_641013010_testa.zip)

下载后的文件：
```
Mode                LastWriteTime         Length Name
----                -------------         ------ ----
-a----        2019/4/17     20:18       41072750 ad_operation.dat
-a----        2019/4/17     20:16       26904219 ad_static_feature.out
-a----        2019/4/17     19:12     3000635881 imps_log.zip
-a----        2019/4/17     20:37        9042342 test_sample.dat
-a----        2019/4/17     19:06     1065378856 user.zip
```


### 历史曝光日志数据文件 totalExposureLog.out
>注：为避免数据量过大，历史曝光数据选择了在用户维度（uv）上按 512 分之一进行均匀采样。各字段使用制表符(\t)分隔，每列的具体含义如下：

|字段名|字段含义|
|----|----|
|广告请求 id|唯一标识每次请求（每个请求对应一个用户某一时刻，可能多个4广告位）|
|广告请求时间|该字段为时间戳，即 1970 纪元后经过的浮点秒数|
|广告位 id|加密后无业务含义，只区分不同广告位，每个广告位只能曝光特定素材尺寸的广告|
|用户 id（即看广告的人）|加密后无业务含义，只区分不同用户，可和后面的用户特征数据中 id 相关联|
|曝光广告 id|加密后无业务含义，只区分不同广告，可以和广告特征文件中的广告 id 关联|
|曝光广告素材尺寸|枚举型取值，不同广告位对素材的尺寸要求不同，同一个广告位可能适配多个不同尺寸的素材|
|曝光广告出价 bid|这里只记录 cpc 出价，非 cpc 广告此处记录折算后的 cpc价格|
|曝光广告 pctr|预估的 pctr，和 bid 相乘得到 basic_ecpm|
|曝光广告 quality_ecpm|将广告质量和用户体验等因素折算成 ecpm 的分数，主要影响因素有 pctr/pcvr/窄定向等|
|曝光广告 totalEcpm|广告排序的分数依据，由 basic_ecpm 和 quality_ecpm相加得到|


### 用户特征属性文件 user_data
>注：用户特征字段未知的均使用 0 表示。每列特征取值都用加密后的 id 表示，均为随机映射。不同列的 id 取值区间会重复。各字段使用制表符(\t)分隔，每列的具体业务含义如下：

|字段名|字段含义|
|----|----|
|用户 id|此处和上面曝光日志文件中的用户 id 关联|
|年龄（Age）|每个取值随机映射为[1-N]的唯一 id|
|性别(Gender)|男/女|
|地域(area)|每个省/市用唯一 id 标识，可能多标签，使用逗号分隔不同 id|
|婚恋状态（Status）|单身/已婚等状态，可能去多值，使用逗号分隔|
|学历(Education)|博士/硕士/本科/高中/初中/小学|
|消费能力（ConsuptionAbility）|高/低|
|设备（device）|IOS/Android, 不区分版本号|
|工作状态（work）|在校大学生/商旅人士/政府公职人员/科研教育者/ IT 互联网工作者/医护工作者, 可能取多值，逗号分隔|
|连接类型(ConnectionType)|无线/2G/3G/4G|
|行为兴趣(behavior)|每个兴趣点一个 id，可多值，逗号分隔|


### 广告数据文件
>广告是指广告主创建的广告创意（或称广告素材）及广告展示相关设置，包含广告的基本信息（广告名称、投放时间等）、广告的推广目标、投放平台、投放的广告规格、所投放的广告创意、广告的受众（即广告的定向设置）以及广告出价等信息。这里将广告的数据分为两部分，一部分是广告静态数据（如所属账户等），另一部分是动态数据，即修改操作数据，主要包括广告状态、出价、定向等可修改的设置，该部分数据对应一个时间戳，多次修改的同一条广告会对应多条记录。

1. 广告静态数据 ad_static_feature.out
>该类广告属性一般从广告创建后无法修改。所有 id 类数据均为加密后随机映射。各列用制表符分隔，含义如下：

|字段名|字段含义|
|----|----|
|广告 id|和曝光日志中的广告 id 相关联|
|创建时间|广告创建时的时间戳|
|广告账户 id|广告所在账户的唯一标识，账户结构分为四级(账户——推广计划——广告——素材)|
|商品 id|广告推广目标的唯一标识，若推广目标是落地页，则该字段为空|
|商品类型|广告推广目标的类型，枚举型|
|广告行业 id|广告所属的行业类别标识|
|素材尺寸|不同广告位对素材的尺寸要求不同，同一个广告可能有多个不同尺寸的素材，用逗号分隔|

2. 广告操作数据 ad_operation.dat
>记录广告操作的流水数据，以及操作后的属性值。各列使用制表符分隔，含义如下：

|字段名|字段含义|
|----|----|
|广告 id|和曝光日志中的广告 id 相关联|
|创建/修改时间|即广告创建或者修改设置的时间|
|操作类型|1-修改，2-新建|
|修改字段|1-广告状态，2-出价，3-人群定向，4-广告时段设置|


|操作|操作后的字段值|
|----|----|
|xxxxxxxxxxxxx||
|广告状态取值|1- 正常，0-失效|
|出价|整数（单位分）|
|投放时段|字符串。包含 7 个 64 位无符号整型数字（逗号分隔），每个整数分别代表周一到周日的投放时段。该整数转为 2 进制后从低到高 48 位bit 代表全天各时段（半小时为一时间窗口）是否投放，1-投放，0-不投。举例说明，17179865088= 1111111111111111111111000000000000，代表投放时段为 6：00-17：00；281474976710655=111111111111111111111111111111111111111111111111，代表全天投放。|
|人群定向|`字符串。格式如下：feature_name1:feature_value1,feature_value2|feature_name2:feature_value3,feature_value4|… 此处 feature_name 取值同用户属性文件中的各列属性名，feature_value 取值 id 同用户属性文件中的定义，不同feature 用“|”分隔，不同 feature 取值用逗号分隔。广告通过人群定向的设置来召回对应的用户请求，对应的人群规则：不同 feature_name 是求 交 集 ， 同 一 featurename 下 不 同 的 value 求并集 ， 未 定 义 的feature_name 则表示 该维度 不 限 。 举 例 如：定向 设 置 为age:51,62,73,84|gender:1|area:1,3,5 ; 则表示该广告能被“（年龄 id 为51 或 62 或 73 或 84） 且 （性别取值为 1） 且 （地域取值为 1 或 3或 5）”的用户召回（即在这些用户上有曝光机会）。`|


### 测试数据 test_sample.dat totalExposureLog.out 
>第 n+1 天的待预估广告以及对应的设置。各列由制表符(\t)分隔， 含义说明如下：

- 样本 id
- 广告 id
- 创建时间
- 素材尺寸
- 广告行业 id
- 商品类型
- 商品 id
- 广告账户 id
- 投放时段
- 人群定向
- 出价（单位分）


以上各字段的格式均和训练数据中的广告数据格式一致。
由于要评估出价相关性，故测试数据中一条广告 id 会对应多条不同出价的样本。
为简化问题，测试数据也要求预估指定广告样本在下一个自然日的曝光量。（此处的
曝光量是和训练数据一样 uv 采样后的结果）。为了更准确的评估，此处的测试广告
已经剔除掉下一个自然日期间再次修改的广告（如暂停、修改出价、定向等），即认
为在预估周期内该广告设置保持不变。



## 评估和提交方式

### 准确性指标
$$SMAPE=\frac{1}{n}\sum ^n_{t=1}\frac{\left | F_t-A_t \right |}{\left ( F_t+A_t \right )/2}$$

其中，Ft 为预估的广告曝光值，At 为真实的曝光值。易知，SMAPE 越小越好。
注：测试数据中某个广告只有一个样本的出价是对应线上真实出价，对该样本的预估结
果对比线上真实曝光（采样后）来评估准确性。

### 出价单调相关性指标
由于竞价机制的特性，在广告其他特征不变的前提下，随着出价的提升，预估曝光值也单调提升才符合业务直觉。所以，定义出价单调相关性评估指标如下：对待预估广告 ad, 除出价 bid 外其他设置不变，任意变化 n 个 bid 取值，得对应的 n 个曝光预估值，计算如下该广告出价单调性得分如下：
$$score=\frac{1}{n}\sum ^n_{t=1}\frac{\left ( impo_0-impo_k \right )\left ( bid_0-bid_k \right )}{\left | \left ( impo_0-impo_k \right )\left ( bid_0-bid_k \right ) \right |}$$

### 最终得分 Totalscore
上述两个指标值域和趋势都不同，为了比赛评分简便，会将上述两个指标各自归一化后
再加权求和得到一个最终得分。形式如下：
$$TotalScore=w_1*\left ( 1-\frac{SMPAE}{2} \right )+w_2*\left ( \frac{MonoScore+1}{2} \right)$$

### 提交方式
参赛者提交结果为一个 submission.csv 文件，编码采用无 BOM 的 UTF-8，格式如下：样
本 ID（和提供的测试样本对应），预估日曝光（最多保留 4 位小数）。用逗号分隔字段，无
空格，无表头。
示例如下：
```
1, 230.1234
2, 35.5678
```